In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, roc_auc_score
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split


In [2]:
temp_df = pd.read_csv('Suicide_Detection.csv')

In [3]:
df = temp_df.iloc[:20000]

In [4]:
df.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [5]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

C:\Users\Ulka\AppData\Local\Temp\ipykernel_25628\1160589198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Unnamed: 0'],axis=1, inplace=True)


In [6]:
df.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [7]:
df['text'] = df['text'].apply(lambda x : x.lower())

C:\Users\Ulka\AppData\Local\Temp\ipykernel_25628\3534604001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x : x.lower())


In [8]:
# Download necessary resources from NLTK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ulka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ulka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
# Function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Function to remove stop words
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words]

# Function to tokenize text
def tokenize(text):
    return word_tokenize(text)

# Additional cleaning: remove non-alphabetic tokens and convert to lowercase
def clean_tokens(tokens):
    return [word.lower() for word in tokens if word.isalpha()]

# Apply all steps: remove punctuation, tokenize, clean tokens, and remove stop words
df['text'] = df['text'].apply(
    lambda x: clean_tokens(remove_stopwords(tokenize(remove_punctuation(x))))
)

# Output the cleaned and tokenized DataFrame
df.head()

C:\Users\Ulka\AppData\Local\Temp\ipykernel_25628\2078368733.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(


,text,class
0,"[ex, wife, threatening, suiciderecently, left,...",suicide
1,"[weird, dont, get, affected, compliments, comi...",non-suicide
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide
3,"[need, helpjust, help, im, crying, hard]",suicide
4,"[losthello, name, adam, struggling, years, afr...",suicide


In [10]:
# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Function to apply stemming
def stem_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]

# Apply the cleaning functions and stemming to the 'tokenized_review' column
df['text'] = df['text'].apply(stem_tokens)

# Output the cleaned and stemmed DataFrame
df.head()


C:\Users\Ulka\AppData\Local\Temp\ipykernel_25628\847003611.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(stem_tokens)


,text,class
0,"[ex, wife, threaten, suiciderec, left, wife, g...",suicide
1,"[weird, dont, get, affect, compliment, come, s...",non-suicide
2,"[final, almost, never, hear, bad, year, ever, ...",non-suicide
3,"[need, helpjust, help, im, cri, hard]",suicide
4,"[losthello, name, adam, struggl, year, afraid,...",suicide


In [11]:
# Join the tokenized lists into strings for CountVectorizer
df['text'] = df['text'].apply(lambda tokens: ' '.join(tokens))


C:\Users\Ulka\AppData\Local\Temp\ipykernel_25628\681359426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda tokens: ' '.join(tokens))


In [12]:
X = df.iloc[:,0:1]
y = df['class']

In [13]:
X

,text
0,ex wife threaten suiciderec left wife good che...
1,weird dont get affect compliment come someon k...
2,final almost never hear bad year ever swear fu...
3,need helpjust help im cri hard
4,losthello name adam struggl year afraid past y...
...,...
19995,wish wasnt like thisi read lot post today comm...
19996,futur druggiewhi see reason live get high fun ...
19997,tri kill helpi tri kill sat plastic bag tape h...
19998,someon downvot post piec shit boyfriend confir...


In [14]:
y

0            suicide
1        non-suicide
2        non-suicide
3            suicide
4            suicide
            ...     
19995        suicide
19996        suicide
19997        suicide
19998        suicide
19999    non-suicide
Name: class, Length: 20000, dtype: object

In [15]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [16]:
#suicide=1, non-suicide=0
y

array([1, 0, 0, ..., 1, 1, 0])

In [17]:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [18]:
X_train.shape

(16000, 1)

In [19]:
X_train['text'].iloc[1]

'think im go insanemi mom hoard go school cant move clutter make hard focu anyth rememb went kitchen saw clutter filth counter breath heartrat went errat fell floor vision almost appear alter everyth around look stretch tower spend night pace morn feel like im go insan pleas help'

In [20]:
# For unigrams and bigrams
cv2 = CountVectorizer(ngram_range=(1, 2), max_features=5000)  # This includes both unigrams and bigrams  #20000 overall accuracy increased
X_train_bow2 = cv2.fit_transform(X_train['text']).toarray()
X_test_bow2 = cv2.transform(X_test['text']).toarray()

X_train_bow2.shape
gnb2 = GaussianNB()

gnb2.fit(X_train_bow2,y_train)
y_pred = gnb2.predict(X_test_bow2)

accuracy_score(y_test,y_pred)


0.83975

In [21]:
print(X_test_bow2)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [22]:
X_train_bow2.shape

(16000, 5000)

In [23]:
confusion_matrix(y_test,y_pred)

array([[1774,  263],
       [ 378, 1585]], dtype=int64)

In [24]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [25]:
y_probs = gnb2.predict_proba(X_test_bow2)[:, 1]  # Probability for the positive class
roc_auc = roc_auc_score(y_test, y_probs)

In [26]:
# Print results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'AUC-ROC: {roc_auc:.2f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.84
Precision: 0.86
Recall: 0.81
F1 Score: 0.83
AUC-ROC: 0.85
              precision    recall  f1-score   support

           0       0.82      0.87      0.85      2037
           1       0.86      0.81      0.83      1963

    accuracy                           0.84      4000
   macro avg       0.84      0.84      0.84      4000
weighted avg       0.84      0.84      0.84      4000



In [207]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Function to preprocess a single text entry
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words and apply stemming
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [stemmer.stem(word.lower()) for word in tokens if word.isalpha() and word.lower() not in stop_words]
    
    # Join the cleaned tokens into a single string
    return ' '.join(cleaned_tokens)

# Example usage with a new text entry
new_data = "I've tried to reach out for help, but it feels like no one understands.".lower()

# Preprocess the new data
new_data = preprocess_text(new_data)

# Output the cleaned text
print(new_data)


ive tri reach help feel like one understand


In [209]:
# Create a DataFrame for the new data
data1 = pd.DataFrame({'text': [new_data]})

# Transform the new data using the fitted vectorizer
X1 = cv2.transform(data1['text']).toarray()  # Correctly access the 'text' column
X1.shape


(1, 5000)

In [211]:
X1.shape


(1, 5000)

In [73]:
# Make predictions on the new data
new_predictions = gnb2.predict(X1)
print(new_predictions)
# Display the results
for text, prediction in zip(data1['text'], new_predictions):
    label = "suiside" if prediction == 1 else "non-suicide"
    print(f'Text: "{text}" - Predicted Label: {label}')


[0]
Text: "feel like there way im tire fight" - Predicted Label: non-suicide


In [75]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow2,y_train)
y_pred = rf.predict(X_test_bow2)
accuracy_score(y_test,y_pred)

0.86975

In [85]:
confusion_matrix(y_test,y_pred)

array([[1741,  296],
       [ 225, 1738]], dtype=int64)

In [87]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [89]:
# Print results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

print(classification_report(y_test, y_pred))

Accuracy: 0.87
Precision: 0.85
Recall: 0.89
F1 Score: 0.87
              precision    recall  f1-score   support

           0       0.89      0.85      0.87      2037
           1       0.85      0.89      0.87      1963

    accuracy                           0.87      4000
   macro avg       0.87      0.87      0.87      4000
weighted avg       0.87      0.87      0.87      4000



In [213]:
# Make predictions on the new data
new_predictions = rf.predict(X1)
print(new_predictions)

# Make predictions on the new data
new_probabilities = rf.predict_proba(X1)

# Display the results
for text, probabilities in zip(data1['text'], new_probabilities):
    # Extract the probability of the positive class (class 1)
    probability_suicide = probabilities[1]  # Probability of class 1 (suicide)
    label = "suicide" if probability_suicide >= 0.5 else "non-suicide"
    
    # Print text with predicted label and probability
    print(f'Text: "{text}" - Predicted Label: {label}, Probability of Suicide: {probability_suicide:.4f}')





# # Display the results
# for text, prediction in zip(data1['text'], new_predictions):
#     label = "suiside" if prediction == 1 else "non-suicide"
#     print(f'Text: "{text}" - Predicted Label: {label}')


[0]
Text: "ive tri reach help feel like one understand" - Predicted Label: non-suicide, Probability of Suicide: 0.4400


Everything feels hopeless, and I can't take it anymore.
I just want the pain to stop; I don’t want to live like this.
I don't see any reason to keep going anymore.
I feel completely alone, and nothing seems to matter anymore
Every day is a struggle, and I don't know how much longer I can endure this pain.
I wish I could escape from this life and find peace.



Sometimes I wish I could just disappear.
I feel like there's no way out and I'm tired of fighting.
The weight of my sadness feels unbearable, and I can't see a way out.
Sometimes, I think everyone would be better off without me. .3
It's hard to find joy in anything when all I feel is despair .2
I've tried to reach out for help, but it feels like no one understands.  .4